In [2]:
# Third party imports
from IPython.display import display, HTML
# Local imports
from libcomcat.dataframes import (get_detail_data_frame, get_dyfi_data_frame,
                                  get_history_data_frame, get_magnitude_data_frame,
                                  get_pager_data_frame, get_phase_dataframe,
                                  get_summary_data_frame)
from libcomcat.search import search, get_event_by_id


In [3]:
import pandas as pd

(Returns NoneType if no pager product)

In [5]:
df = pd.read_csv('../DASHBOARD/CSV_ORIGINAL/raw_alert_usa.csv')

In [6]:
df.head(1)

,type,id,properties.mag,properties.place,properties.time,properties.updated,properties.tz,properties.url,properties.detail,properties.felt,...,properties.rms,properties.gap,properties.magType,properties.type,properties.title,geometry.type,geometry.coordinates,Longitud,Latitud,Profundidad
0,Feature,nc73887046,5.16,"4km NW of Canyondam, CA",2023-05-12 10:18:41.310,2023-05-15 14:53:47.275,NaN,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,1347.0,...,0.19,37.0,mw,earthquake,"M 5.2 - 4km NW of Canyondam, CA",Point,"[-121.0998333, 40.196, 6.06]",-121.099833,40.196,6.06


In [10]:
import json

In [ ]:
extra_data = []
for id in df['id']:
    pager_event = get_event_by_id(id)
    pager_country_df = get_pager_data_frame(pager_event, get_losses=True)
    
    if pager_country_df is not None:
        try:
            pager_country_json = pager_country_df.to_json()
            pager_country_data = json.loads(pager_country_json)
            extra_data.append(pager_country_data)
        except json.JSONDecodeError as e:
            print(f"Error decoding JSON for id {id}: {e}")
            continue

extra_data_df = pd.concat(extra_data)

df = pd.merge(df, extra_data_df, on='id', how='left')
df.head(1)